In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import datetime



import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yfin
yfin.pdr_override()


import warnings

In [ ]:
pd.set_option('display.float_format', lambda x: f'{x:,.6f}')
plt.rcParams["figure.figsize"] = (10,6)

#sk.set_config(display='diagram')
warnings.filterwarnings("ignore")

# Assignment

## Variables

In [9]:
start = datetime.datetime(2022, 10, 1)
split_date = datetime.datetime(2023,8,24)
today = datetime.datetime(2023,10,20)

## Functions

# Load Data

In [ ]:
folio_df = pd.read_csv('https://raw.githubusercontent.com/dsimband/DATA618/main/w12/data/DATA618_Portfolio.csv', 
                 dtype={
                    'ID': 'int',
                    'Price': 'float',
                    'Shares': 'float',
                    'Value': 'float', 
                 })

folio_df = folio_df[folio_df['Shares'] > 0]
folio_df = folio_df.groupby(['Ticker','BondName','Class','Sub_Class'])[['Shares','Value']].sum()
folio_df.reset_index(inplace=True)

# calculate portfilio percentage
portfolio_total = folio_df['Value'].sum()
folio_df['port_percent'] = folio_df['Value'] / portfolio_total

# Class Portfolios
folio_econ_df = folio_df[folio_df['Class'] == 'Economically Sensitive']
folio_int_df = folio_df[folio_df['Class'] == 'Interest Rate Sensitive']

folio_df